In [1]:
import pandas as pd
import re
import string
import sklearn
import csv

In [2]:
df = pd.read_csv("dataset3.csv")
file = open('synopsis_shawshank.txt',"r")
#df2 = pd.read_csv("synopsis2.csv")
text=file.read()
#print(text)
sentences = text.split('. ')
#print(sentences)
    
df2 = pd.DataFrame(sentences, columns=["Positive"])
df2.to_csv('synopsis3.csv', index=False)
print(df.tail())

      Unnamed: 0  is_spoiler  \
4356        4356       False   
4357        4357       False   
4358        4358       False   
4359        4359       False   
4360        4360       False   

                                            review_text Unnamed: 3 Unnamed: 4  \
4356  Some movies really need more than 2 hours to d...        NaN        NaN   
4357  With all due respect to the people that compla...        NaN        NaN   
4358  Every single scene in The Shawshank Redemption...        NaN        NaN   
4359  At the beginning i thought, this would be an a...        NaN        NaN   
4360  In years to come the Shawshank Redemtion will ...        NaN        NaN   

     Unnamed: 5  
4356        NaN  
4357        NaN  
4358        NaN  
4359        NaN  
4360        NaN  


In [3]:
k = 700 # no of comments in training positive examples.

df3 = pd.DataFrame(columns=["review_text","is_spoiler"])
spoiler_comments = []
cnt = 0
for i in range(len(df.index)):
    if(df.iloc[i]["is_spoiler"]==0):
        df3 = df3.append(df.iloc[i],ignore_index = True)
    else:
        if cnt<k:
            spoiler_comments.append((df.iloc[i])["review_text"])
            cnt+=1
        else :
            df3 = df3.append(df.iloc[i],ignore_index = True)
            
temp_df = pd.DataFrame(spoiler_comments, columns=["Positive"])
df2 = df2.append(temp_df,ignore_index = True)

""""print(df2.head())
print(df3.head())  """ 
print(df2.tail())   # train
print(df3.tail())   # test

                                              Positive
837  So, i watched this film thinking it'd be prett...
838  I was not born when this movie came out so I h...
839  This is a movie that does not try to outsmart ...
840  There aren't words too describe the greatness ...
841  Adapted from a Stephen King novella, The Shaws...
                                            review_text is_spoiler  \
3656  Some movies really need more than 2 hours to d...      False   
3657  With all due respect to the people that compla...      False   
3658  Every single scene in The Shawshank Redemption...      False   
3659  At the beginning i thought, this would be an a...      False   
3660  In years to come the Shawshank Redemtion will ...      False   

      Unnamed: 0 Unnamed: 3 Unnamed: 4 Unnamed: 5  
3656      4356.0        NaN        NaN        NaN  
3657      4357.0        NaN        NaN        NaN  
3658      4358.0        NaN        NaN        NaN  
3659      4359.0        NaN        NaN   

In [4]:
index=[]
for i in range(len(df2.index)):
    index.append("Positive "+str(i))
#df.insert(loc=0,column='index',value=index)
df2['index']=index
df2.set_index('index', inplace=True)

index2=[]
for i in range(len(df3.index)):
    index2.append("Comment "+str(i))
#df.insert(loc=0,column='index',value=index)
df3['index']=index2
df3.set_index('index', inplace=True)
df3

,review_text,is_spoiler,Unnamed: 0,Unnamed: 3,Unnamed: 4,Unnamed: 5
index,,,,,,
Comment 0,Few movies can get this critic watching them a...,True,700.0,NaN,NaN,NaN
Comment 1,why i think this is the best movie ever!!! ......,True,701.0,NaN,NaN,NaN
Comment 2,"Hope can set anyone free, imprisoned or not im...",True,702.0,NaN,NaN,NaN
Comment 3,"With the many other entries already here, I wo...",True,703.0,NaN,NaN,NaN
Comment 4,The Shawshank Redemption is a great movie. You...,True,704.0,NaN,NaN,NaN
Comment 5,This film captivates the imagination and gives...,True,705.0,NaN,NaN,NaN
Comment 6,I have no words to express the feelings I got ...,True,706.0,NaN,NaN,NaN
Comment 7,I was initially sceptical about watching this ...,True,707.0,NaN,NaN,NaN
Comment 8,This movie is absolute perfection. The story i...,True,708.0,NaN,NaN,NaN


In [5]:
def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text
round1 = lambda x: clean_text_round1(x)

In [6]:
data_clean = pd.DataFrame(df2.Positive.apply(round1))
data_clean2 = pd.DataFrame(df3.review_text.apply(round1))
data_clean2

,review_text
index,
Comment 0,few movies can get this critic watching them a...
Comment 1,why i think this is the best movie ever i thi...
Comment 2,hope can set anyone free imprisoned or not imp...
Comment 3,with the many other entries already here i won...
Comment 4,the shawshank redemption is a great movie you ...
Comment 5,this film captivates the imagination and gives...
Comment 6,i have no words to express the feelings i got ...
Comment 7,i was initially sceptical about watching this ...
Comment 8,this movie is absolute perfection the story is...


In [7]:
data_clean

,Positive
index,
Positive 0,in andy dufresne tim robbins a banker in main...
Positive 1,since the state of maine has no death penalty ...
Positive 2,andy keeps claiming his innocence but his cold...
Positive 3,despite his best efforts and behavior reds par...
Positive 4,red is then introduced as the local smuggler w...
Positive 5,an alarm goes off alerting all prisoners of ne...
Positive 6,red and his friends bet on whichever new fish ...
Positive 7,red places a huge bet on andy\n\nduring the fi...
Positive 8,however the celebration is short lived when th...


In [8]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(data_clean.Positive)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_clean.index


cv2 = CountVectorizer(stop_words='english')
data_cv2 = cv2.fit_transform(data_clean2.review_text)
data_dtm2 = pd.DataFrame(data_cv2.toarray(), columns=cv2.get_feature_names())
data_dtm2.index = data_clean2.index
data_dtm2

,aa,aaaaaaa,aaahhh,aback,abandon,abated,abbreviation,abc,abd,abetted,...,zone,zonethis,zoolander,zoomout,zooms,½review,ándale,été,ëfail,ëvideomalaiseí
index,,,,,,,,,,,,,,,,,,,,,
Comment 0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Comment 1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Comment 2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Comment 3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Comment 4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Comment 5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Comment 6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Comment 7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Comment 8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# LDA
import numpy as np
from gensim import matutils, models
import scipy.sparse

In [12]:
total_topics=20
lda = models.LdaMulticore(corpus=corpus, id2word=id_word_dict, num_topics=total_topics, passes=50)
#lda.print_topics()

In [14]:
word_weight_positive={}
cnt=0
for i in range(total_topics):
    word_topic=lda.print_topic(i).split('+')
    j=0
    for word_probab in word_topic:
        val,wor=word_probab.split('*')        
        wor=str(wor)
        #print(wor)
        y=len(wor)
        if j==len(word_topic)-1:
            wor = wor[1:y-1]
        else:
            wor = wor[1:y-2]
        #print(wor)
        if wor not in word_weight_positive:
            word_weight_positive[wor]=cnt
            cnt+=1
            
        
        j=j+1

In [15]:
word_weight_positive

{'film': 0,
 'life': 1,
 'small': 2,
 'great': 3,
 'history': 4,
 'hope': 5,
 'story': 6,
 'best': 7,
 'cinema': 8,
 'powerful': 9,
 'movie': 10,
 'good': 11,
 'watch': 12,
 'like': 13,
 'prison': 14,
 'movies': 15,
 'time': 16,
 'place': 17,
 'cu': 18,
 'si': 19,
 'harry': 20,
 'weight': 21,
 'franny': 22,
 'james': 23,
 'long': 24,
 'andy': 25,
 'red': 26,
 'shawshank': 27,
 'years': 28,
 'hes': 29,
 'redemption': 30,
 'robbins': 31,
 'lot': 32,
 'suggestion': 33,
 'cool': 34,
 'funny': 35,
 'love': 36,
 'morgan': 37,
 'characters': 38,
 'does': 39,
 'didnt': 40,
 'man': 41,
 'wife': 42,
 'warden': 43,
 'audience': 44,
 'capability': 45,
 'display': 46,
 'art': 47,
 'tommy': 48,
 'common': 49,
 'nortons': 50,
 'pro': 51,
 'makes': 52,
 'ability': 53,
 'people': 54,
 'darabont': 55,
 'films': 56,
 'freeman': 57,
 'andys': 58,
 'just': 59,
 'character': 60,
 'tim': 61,
 'brooks': 62,
 'camera': 63,
 'scene': 64,
 'think': 65,
 'cadet': 66,
 'tonight': 67,
 'celebrate': 68,
 'prisoners'

In [28]:
from nltk.corpus import wordnet
nltk.download('wordnet')
print(lda.print_topic(5))

0.015*"movie" + 0.013*"movies" + 0.012*"robbins" + 0.010*"lot" + 0.009*"suggestion" + 0.009*"cool" + 0.009*"funny" + 0.009*"love" + 0.007*"prison" + 0.005*"morgan"


[nltk_data] Downloading package wordnet to
[nltk_data]     /home/shivashish/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [29]:
train_size = len(df2.index)
test_size = len(df3.index)
no_of_words = len(word_weight_positive)
X_train = np.zeros((train_size,no_of_words))
X_test = np.zeros((test_size,no_of_words))
Y_test = np.zeros(test_size)

In [31]:
for i in range(train_size):
    if i%10 == 0:
        print(i)
    for j in range(data_dtm.shape[1]):
        if data_dtm.columns[j] not in word_weight_positive:
            flag=0
            for syn in wordnet.synsets(data_dtm.columns[j]): 
                for l in syn.lemmas(): 
                    temp_word = l.name()
                    if temp_word in word_weight_positive:
                        flag=1
                        break
                if flag==1:
                    break
            if flag==1:
                index = word_weight_positive[temp_word]
                X_train[i][index]+=data_dtm.iloc[i][j]
        else :
            index = word_weight_positive[data_dtm.columns[j]]
            X_train[i][index]+=data_dtm.iloc[i][j]

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840


In [35]:
for i in range(test_size):
    if i%20 == 0:
        print(i)
    for j in range(data_dtm2.shape[1]):
        if data_dtm2.columns[j] not in word_weight_positive:
            flag=0
            for syn in wordnet.synsets(data_dtm2.columns[j]): 
                for l in syn.lemmas(): 
                    temp_word = l.name()
                    if temp_word in word_weight_positive:
                        flag=1
                        break
                if flag==1:
                    break
            if flag==1:
                index = word_weight_positive[temp_word]
                X_test[i][index]+=data_dtm2.iloc[i][j]
        else :
            index = word_weight_positive[data_dtm2.columns[j]]
            X_test[i][index]+=data_dtm2.iloc[i][j]

0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020
1040
1060
1080
1100
1120
1140
1160
1180
1200
1220
1240
1260
1280
1300
1320
1340
1360
1380
1400
1420
1440
1460
1480
1500
1520
1540
1560
1580
1600
1620
1640
1660
1680
1700
1720
1740
1760
1780
1800
1820
1840
1860
1880
1900
1920
1940
1960
1980
2000
2020
2040
2060
2080
2100
2120
2140
2160
2180
2200
2220
2240
2260
2280
2300
2320
2340
2360
2380
2400
2420
2440
2460
2480
2500
2520
2540
2560
2580
2600
2620
2640
2660
2680
2700
2720
2740
2760
2780
2800
2820
2840
2860
2880
2900
2920
2940
2960
2980
3000
3020
3040
3060
3080
3100
3120
3140
3160
3180
3200
3220
3240
3260
3280
3300
3320
3340
3360
3380
3400
3420
3440
3460
3480
3500
3520
3540
3560
3580
3600
3620
3640
3660


In [36]:
for i in range(test_size):
    Y_test[i]=df3.iloc[i]["is_spoiler"]

In [37]:
from sklearn.svm import OneClassSVM
lpu_model = OneClassSVM(gamma='auto').fit(X_train)
Y_out = lpu_model.predict(X_test)

In [38]:
wrong=0
for i in range(len(Y_out)):
    if Y_out[i]==-1 and Y_test[i]==1:
        wrong+=1
    else :
        if Y_out[i]==1 and Y_test[i]==0:
            wrong+=1
print(wrong/len(Y_out))

0.6066648456705818
